In [142]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn
import pathlib as pl

In [143]:
dictionarypath = '../dataset/dictionary/Loughran-McDonald_MasterDictionary_1993-2021.csv'
wds_dict = pd.read_csv(dictionarypath)
positive_wds = [row['Word'].lower() for i, row in wds_dict.iterrows() if row['Positive']!=0]
negative_wds = [row['Word'].lower() for i, row in wds_dict.iterrows() if row['Negative']!=0]

In [144]:
path = '../dataset/finRep/FinancialPhraseBank/all-data.csv'
df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])
# print(df[:10])

In [145]:
# to install keybert
# !pip install nltk
# !pip install keybert
# !pip install sentence-transformers
# !pip install git+https://github.com/LIAAD/yake
### To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [146]:
# choose a random sample
text = df.sample(1)['sentence'].iloc[0]
label = df.sample(1)['sentiment'].iloc[0]
# text = "you are not good at trading "
print(text)
print("the sentiment is : %s"%label)

Net interest income totaled EUR 15.9 mn , compared to EUR 15.6 mn a year earlier .
the sentiment is : negative


In [147]:
def normalize_keyword_list(keyword_list):
    keys = []
    for keyword in keyword_list:
        keys.append(keyword[0])
    return keys

Rake

In [148]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
rake_keywords = rake_nltk_var.get_ranked_phrases()
print(rake_keywords)

['net interest income totaled eur 15', 'eur 15', 'year earlier', '9 mn', '6 mn', 'compared']


In [149]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [150]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [151]:
# print(f'Text: {text}.\nLength: {len(text.split())}')
keybert_keywords = extract_terms(text)
keybert_keywords = normalize_keyword_list(keybert_keywords)

print(keybert_keywords)

['income totaled eur', 'mn year earlier', '15', 'net', 'compared']


In [152]:
import yake
kw_extractor = yake.KeywordExtractor()
yake_keywords = kw_extractor.extract_keywords(text)
yake_keywords = normalize_keyword_list(yake_keywords)

print(yake_keywords)


# print("SECOND FORM:")
# language = "en"
# max_ngram_size = 3
# deduplication_threshold = 0.9
# deduplication_algo = 'seqm'
# windowSize = 1
# numOfKeywords = 20

# custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
# keywords = custom_kw_extractor.extract_keywords(text)

# for kw in keywords:
#     print(kw)

['income totaled EUR', 'Net interest income', 'interest income totaled', 'totaled EUR', 'Net interest', 'year earlier', 'compared to EUR', 'interest income', 'income totaled', 'EUR', 'Net', 'compared', 'earlier', 'interest', 'income', 'totaled', 'year']


In [153]:
Keywords = {}
Keywords["rake"] = rake_keywords
Keywords["keybert"] = keybert_keywords
Keywords["yake"] = yake_keywords
print("Rake: %s" % Keywords["rake"]) 
print("Keybert: %s" % Keywords["keybert"]) 
print("Yake: %s" % Keywords["yake"]) 

Rake: ['net interest income totaled eur 15', 'eur 15', 'year earlier', '9 mn', '6 mn', 'compared']
Keybert: ['income totaled eur', 'mn year earlier', '15', 'net', 'compared']
Yake: ['income totaled EUR', 'Net interest income', 'interest income totaled', 'totaled EUR', 'Net interest', 'year earlier', 'compared to EUR', 'interest income', 'income totaled', 'EUR', 'Net', 'compared', 'earlier', 'interest', 'income', 'totaled', 'year']


NLTK-VADER

In [154]:
print(keybert_keywords)

['income totaled eur', 'mn year earlier', '15', 'net', 'compared']


In [155]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

result = []
for sentence in keybert_keywords:
    s = sentence.split()
    keyword_sen = []
    for word in s:
        if (sid.polarity_scores(word)['compound']) >= 0.5:
            keyword_sen.append(tuple((word, 1)))
        elif (sid.polarity_scores(word)['compound']) <= -0.5:
            keyword_sen.append(tuple((word, -1)))
        else:
            keyword_sen.append(tuple((word, 0)))
    result.append(keyword_sen)

print(result)              

[[('income', 0), ('totaled', 0), ('eur', 0)], [('mn', 0), ('year', 0), ('earlier', 0)], [('15', 0)], [('net', 0)], [('compared', 0)]]


Using TextBlob

In [156]:
from textblob import TextBlob

result = []
for sentence in keybert_keywords:
    s = sentence.split()
    keyword_sen = []
    for word in s:
        testimonial = TextBlob(word)
        if testimonial.sentiment.polarity >= 0.5:
            keyword_sen.append(tuple((word, 1)))
        elif testimonial.sentiment.polarity <= -0.5:
            keyword_sen.append(tuple((word, -1)))
        else:
            keyword_sen.append(tuple((word, 0)))
    result.append(keyword_sen)
print(result)


[[('income', 0), ('totaled', 0), ('eur', 0)], [('mn', 0), ('year', 0), ('earlier', 0)], [('15', 0)], [('net', 0)], [('compared', 0)]]


Using dictionary

In [157]:
def compare_words(keyword_list):
    result = []
    for sentence in keyword_list:
        words = sentence.split()
        keyword_sen = []
        for word in words:
            if word in positive_wds:
                keyword_sen.append(tuple((word, 1)))
            elif word in negative_wds:
                keyword_sen.append(tuple((word, -1)))
            else:
                keyword_sen.append(tuple((word, 0)))
        result.append(keyword_sen)
    return result

In [158]:
keybert_sen = compare_words(keybert_keywords) #keybert

print(keybert_sen)

[[('income', 0), ('totaled', 0), ('eur', 0)], [('mn', 0), ('year', 0), ('earlier', 0)], [('15', 0)], [('net', 0)], [('compared', 0)]]


In [159]:
# !pip install spacy
# !python3 -m spacy download en_core_web_sm

In [160]:
import spacy
nlp = spacy.load('en_core_web_sm')
FinalKeys = []
for keybert in Keywords["keybert"]:
    word1 = nlp(keybert)
    for rake in Keywords["rake"]:
        word2 = nlp(rake)
        for yake in Keywords["yake"]:
            word3 = nlp(yake)
            if word3.similarity(word1) > 0.7 and (not(keybert in FinalKeys)):
                FinalKeys.append(keybert)
            if word3.similarity(word2) > 0.7 and not(rake in FinalKeys):
                FinalKeys.append(rake)
            if word2.similarity(word1) > 0.7 and not(keybert in FinalKeys):
                FinalKeys.append(keybert)
print(FinalKeys)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_

['income totaled eur', 'net interest income totaled eur 15', 'year earlier', 'compared', 'mn year earlier', '15']
